# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import scipy
import os
import glob
import pandas as pd
import math
from datetime import datetime, timedelta
import collections
plt.rcdefaults() #Setting default color of output plots
# For interactive plots

plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib widget

# TODO:
    
   1) GRB ID SHOULD BE USED AS INPUT IN MOST FUNCTIONS!! IMPLEMENT THIS 
   2) Correct concatination of multiple files. GRB_ID is used to locate list of files
   2) Use GRBID as input in plotting function, not filename. Don't need this right now tho. FOR LED AND HED
   3) Redo all the plots ...

# Paths

In [32]:
path_HED = "/Home/siv30/wad005/master/GRB/MATLAB/output/file_mat_TGF/"              #UiB
path_LED = "/Home/siv30/wad005/master/GRB/MATLAB/CZT_calibration/file_mat_CZT_TGF/" #UiB
#path_HED = "/Users/andreas/phys/master/GRB/MATLAB/output/file_mat_TGF/"               #MAC
#path_LED = "/Users/andreas/phys/master/GRB/MATLAB/CZT_calibration/file_mat_CZT_TGF/"  #MAC

# Dict creation

In [3]:
trig_dict_HED = {"GRB180720":["2018_07_20_14_21_48","2018_07_20_14_21_51","2018_07_20_14_21_56","2018_07_20_14_21_57","2018_07_20_14_22_09"],
                 "GRB181222":["2018_12_22_20_11_36"],
                 "GRB190206":["2019_02_06_03_49_28"],
                 "GRB190305":["2019_03_05_13_05_19"],
                 "GRB190501":["2019_05_01_05_23_22"],
                 "GRB190606":["2019_06_06_01_55_07"],
                 "GRB190615":["2019_06_15_14_42_21"],
                 "GRB190813":["2019_08_13_12_00_49"],
                 "GRB190825":["2019_08_25_21_04_55","2019_08_25_21_04_53"], 
                 "GRB190829":["2019_08_29_19_56_40"],
                 "GRB190906":["2019_09_06_01_04_51"],
                 "GRB191001":["2019_10_01_06_41_52"],
                 "GRB191004":["2019_10_04_18_07_03"],
                 "GRB191119":["2019_11_19_10_41_06"],
                 "GRB191221":["2019_12_21_20_39_10","2019_12_21_20_39_11","2019_12_21_20_39_12","2019_12_21_20_39_13","2019_12_21_20_39_15"],
                 "GRB200412":[],# long list
                 "GRB200521":["2020_05_21_12_16_39"],
                 "GRB200716":["2020_07_16_22_57_40"],
                 "GRB200925":["2020_09_25_21_50_31"],
                 "GRB201227":["2020_12_27_15_14_06"]
            }
trig_dict_HED = collections.OrderedDict(sorted(trig_dict_HED.items())) #Sorting the keys in decending order. Now trig_dict_HED is an OderedDict type

for key in trig_dict_HED.keys():
    trig_dict_HED[key].sort() #sorting files in increasing order


In [4]:
trig_dict_LED = {"GRB181227":["2018_12_27_06_17_04.210500_Fulltrigger","2018_12_27_06_17_09.775992_Fulltrigger","2018_12_27_06_17_10.800851_Fulltrigger"],
                "GRB190117":["2019.01.17_08.50.42.939689_Fulltrigger"],
                 "GRB190420":["2019.04.20_23.32.24.931211_Fulltrigger"],
                 "GRB190501":["2019.05.01_05.23.21.736797_Fulltrigger"],
                 "GRB190615":["2019.06.15_14.42.22.741579_Fulltrigger"],
                 "GRB190628":["2019.06.28_04.23.32.512290_Fulltrigger"],
                 "GRB190813":["2019.08.13_12.00.48.801260_Fulltrigger"],
                 "GRB190825":["2019.08.25_21.04.54.439617_Fulltrigger","2019.08.25_21.04.56.395859_Fulltrigger"],
                 "GRB190906":["2019.09.06_01.04.52.014004_Fulltrigger"],
                 "GRB191001":["2019.10.01_06.41.51.631214_Fulltrigger"],
                 "GRB191004":["2019.10.04_18.07.03.402478_Fulltrigger"],
                 "GRB191119":["2019.11.19_10.41.05.239944_Fulltrigger"],
                 "GRB191221":["2019.12.21_20.39.11.939846_Fulltrigger","2019.12.21_20.39.12.939997_Fulltrigger","2019.12.21_20.39.13.940233_Fulltrigger",
                              "2019.12.21_20.39.15.106045_Fulltrigger","2019.12.21_20.39.17.252498_Fulltrigger"]
                 
                }
trig_dict_LED = collections.OrderedDict(sorted(trig_dict_LED.items())) #Sorting the keys in decending order. Now trig_dict_HED is an OderedDict type

for key in trig_dict_LED.keys():
    trig_dict_LED[key].sort() #sorting files in increasing order


# Functions for handling matlab files

## CZT function

In [5]:
def read_matlab_file_with_CZT_data(filename):
    '''Read matlab file with ASIM CZT data from the low energy detector (LED)

    tus: relative time in microseconds (excluded multihits)
    erg: energy channel (excluded multihits)
    energy_keV: energy in keV (excluded multihits)
    tus_keep_multihit: relative time in microseconds (included multihits)
    erg_keep_multihit: energy channel (included multihits)
    multihit_indx: multihit index 0-4. 0->1 1->2// 0: tus=105, 1: tus=110,110 (two counts)
    '''
    mat = scipy.io.loadmat(path_LED + "%s.mat" % (filename))

    #keV_spectra = mat['keV_spectra_total_fulltrigger']

    # Included multihits. Important for light curves
    tus_keep_multihit = np.array(mat['tus_keep_multihit_fulltrigger'])[:,0] # relative time in microseconds
    erg_keep_multihit = np.array(mat['erg_keep_multihit_fulltrigger'])[0,:] # energy channel (unreliable for multi hits)
    multihit_indx = np.array(mat['multihit_fulltrigger'])[0,:] # multihit index. How many multihits in count. (0=1 count, 1=2 counts, 2=3 counts...)
    
    # The multihits are removed as they don't have a reliable energy estimate
    erg = np.array(mat['erg_fulltrigger'])[0,:] # energy channel of CZT pixel
    tus = np.array(mat['tus_fulltrigger'])[:,0] # relative time in microseconds
    energy_keV = np.array(mat['pixel_energy_fulltrigger']).flatten() # energy in keV of CZT pixel
    date = mat['T_ref_char'][0] #raw datetime
    
    #Logic that checks for array lengths
    try:
        length = len(tus_keep_multihit)
        if any(len(lst) != length for lst in [erg_keep_multihit, multihit_indx]):
        # at least one list has a different length
            raise ValueError("Arrays have different lengths")
            
        t_temp = []
        erg_temp = []
        
        for index, i in enumerate(multihit_indx):
            for j in np.arange(0,i+1): #adding i+1 times the tus_count
                t_temp.append(tus_keep_multihit[index])
                erg_temp.append(erg_keep_multihit[index])
         
        tus_keep_multihit_ntimes =  np.array(t_temp) #array of tus. Multihit index decides how many times the tus is appended
        erg_keep_multihit_ntimes = np.array(erg_temp) #array of erg. Multihit index decides how many times the erg is appended
        
        return tus, erg, energy_keV, tus_keep_multihit_ntimes, erg_keep_multihit_ntimes, multihit_indx, date 
    
    except ValueError as ve:
        print("Function aborted because of: " + str(ve))
        print("Length of arrays: ")
        for lst in [tus_keep_multihit, erg_keep_multihit, multihit_indx]:
            print(len(lst))

## BGO function

In [6]:
def read_matlab_file_with_BGO_data(matlabfilename):
    '''Read matlab file with ASIM BGO data from the high energy detector (HED)

    tus: relative times in microseconds.
    erg: energy channel (needs to be calibrated to keV using calibration coeff from https://git.app.uib.no/ASIM-MXGS/IN-FLIGHT_CALIBRATION_BGO/-/tree/master/CALIBRATION_COEFS )
    typ: normal (0), fast(1), valley (3), or overflow event (2 or 4 not )
    adr: BGO address 1-12
    dau: dau address 1-4
    det: detector address 1-3
    DetCol: RGB color
    
    - Kepping only normal (0) and fast (1) events
    '''
    matlabfile_TGF = path_HED + '%s.mat' % matlabfilename
    
    mat = scipy.io.loadmat(matlabfile_TGF)
    
    tus = mat['tus'].flatten()
    erg = mat['erg'].flatten()
    DetCol = mat['DetCol'] # Investigate this matrix
    adr = mat['adr'].flatten()
    dau = mat['dau'].flatten()
    det = mat['det'].flatten()
    typ = mat['typ'].flatten()
    
    date = mat['date'][0] #raw datetime
    
    
    try:
        length = len(tus)
        if any(len(lst) != length for lst in [erg, typ, adr, dau, det]):
        # at least one list has a different length
            raise ValueError("Arrays have different lengths")
            
        mask = np.where((typ != 0) & (typ != 1)) #finding indecies of vallays and overflow events
        
        # removing those indecies
        tus = np.delete(tus,mask)
        erg = np.delete(erg,mask)
        adr = np.delete(adr,mask)
        dau = np.delete(dau,mask)
        det = np.delete(det,mask)
        typ = np.delete(typ,mask)
        
        
        return tus, erg, typ, adr, dau, det, DetCol, date
    
    except ValueError as ve:
        print("Function aborted because of: " + str(ve))
        print("Length of arrays: ")
        for lst in [tus,erg, typ, adr, dau, det]:
            print(len(lst))

# Plotting functions

## LED

In [7]:
def plot_LED(grb_id=None, LED_filename=None, binsize=10,start_range =0, end_range=0):
    
    try:
        if grb_id != None and LED_filename == None:
            if grb_id not in trig_dict_LED.keys():
                raise KeyError("GRB ID not in dict")
            else:
                LED_filename = trig_dict_LED[grb_id][0]
        else:
            LED_filename = LED_filename
    except KeyError as e:
        return print(e)

    tus_LED, erg_LED, energy_keV_LED, tus_keep_multihit_LED, erg_keep_multihit_LED, multihit_indx_LED, date = read_matlab_file_with_CZT_data(
    LED_filename)

    binsize = binsize*1e3 #converting to microseconds
    start_range = start_range*1e3 #converting to microseconds
    end_range = end_range*1e3
    
    fig0, ax1 = plt.subplots(
        1, 1, figsize=(12, 8), sharex=True, sharey=False)
    
    ax1.hist(tus_keep_multihit_LED, bins=np.arange(
        min(tus_keep_multihit_LED), max(tus_keep_multihit_LED) + binsize, binsize),edgecolor= "black", histtype="step")
    ax1.set_xlabel("time [us], binsize {} ms".format((binsize / 1e3)))
    ax1.set_ylabel("Counts per bin")
    ax1.set_title("GRB " + date)
    
    if start_range and end_range != 0:
        mean = get_mean(LED_filename,binsize,tus_LED, start_range, end_range)
        ax1.axhline(y=mean, color="red", label="Mean of bg: {} counts/bin \n First range [{} - {}] s \n 2nd: [{} - {}] s".format(
            np.round(mean,2), np.round(min(tus_LED)/1e6,3),start_range/1e6, end_range/1e6, np.round(max(tus_LED)/1e6,3)),linestyle='--')
                                                                                                                              
                                                                                                                          
        ax1.legend()
        plt.show()
    else:
        plt.show()
        
    #plt.savefig(output_dir + "/{}_LED".format(grb_id))

    

## HED

In [8]:
def plot_HED(grb_id=None, binsize=10, start_GRB=0, end_GRB=0, fig=(12,8),tus_arr = None, T0=None):
    
    """Plotting data from HED. If HED_filename is not specified, but tus_arr is: plotting the tus array without reading matlab file
        If start and end value for the GRB is not given, the data is plotted normally,
        - BINSIZE IS INITIALIZED AS MILLISECONDS
        - start_GRB & end_GRB IS INITIALIZED AS MILLISECONDS 0.5 s = 500 ms
        - tus_HED is given in microseconds
    """
    try:
        if grb_id not in trig_dict_HED.keys():
            raise KeyError("GRB ID not in dict")
        else:
            HED_filename = trig_dict_HED[grb_id][0]
    except KeyError as e:
        return print(e)
    
    binsize = binsize*1e3 #converting to microseconds\n",
    
    start_GRB = start_GRB*1e3 #converting to microseconds\n",
    end_GRB = end_GRB*1e3,
    
    #Creating switch for input
    
    if HED_filename != None and tus_arr == None:
        tus_HED, erg_HED, typ_HED, adr_HED, dau_HED, det_HED, DetCol_HED, date = read_matlab_file_with_BGO_data(HED_filename)
        T0 = datetime.strptime(date, "%Y-%b-%d %H:%M:%S.%f")
        date = datetime.strftime(T0,format='%y%m%d') #Correct format for title of plot: GRBYYMMDD
        grb_id = "GRB" + date
        
    elif HED_filename == None and type(tus_arr) == np.ndarray and type(T0) == datetime:
        tus_HED = tus_arr
        date = datetime.strftime(T0,format='%y%m%d') #Correct format for title of plot: YYMMDD
        grb_id = "GRB" + date
        
    else:
        return "T0 must be specified when initiating tus_arr"
    
    fig0, ax1 = plt.subplots(
        1, 1, figsize=fig, sharex=True, sharey=False)
    ax1.hist(tus_HED, bins=np.arange(
        min(tus_HED), max(tus_HED) + binsize, binsize),edgecolor= "black", histtype="step")
    
    ax1.set_xlabel("time [us], binsize {} ms".format((binsize / 1e3)))
    ax1.set_ylabel("Counts per bin"),
    ax1.set_title(grb_id) 


    if start_GRB and end_GRB != 0:
        mean = get_mean(HED_filename,binsize,tus_HED, start_GRB, end_GRB)
        ax1.axhline(y=mean, color="red", label= "Mean of bg: {} counts/bin \n First range [{} - {}] s \n 2nd: [{} - {}] s".format(
            np.round(mean,2), np.round(min(tus_HED)/1e6,3),start_GRB/1e6, end_GRB/1e6, np.round(max(tus_HED)/1e6,3)),linestyle='--'),
        ax1.legend()
        plt.show(),
    else:
        plt.show()

## mean

In [9]:
# Rename start_GRB to start_range. Same for end_GRB
def get_mean(matlab_filename, binsize, tus, start_GRB, end_GRB):
    
    """Function for getting the mean from the background.
    matlab_filename: <str> filename
    binsize: <int> given in us // 5ms = 5000 us 
    start & end_GRB given in us.
    -Finding ranges to compute the mean in each bin. """
    
    tus = np.sort(tus)
    range_1 =  find_closest(tus,start_GRB)
    range_2 = find_closest(tus,end_GRB)
    
    range_1_counts = np.histogram(tus[:range_1], bins=np.arange(
        min(tus[:range_1]), max(tus[:range_1]) + binsize, binsize))[0]
    
    range_2_counts = np.histogram(tus[range_2:], bins=np.arange(
        min(tus[range_2:]), max(tus[range_2:]) + binsize, binsize))[0]
    
    background_data = np.append(range_1_counts,range_2_counts)
    mean = np.mean(background_data)
    return mean

In [10]:
def find_nearest(array, value):
    """Returns nearest index for value in array.
    """
    array = np.asarray(array)
    return (np.abs(array - value)).argmin()

In [26]:
a = np.array([1,2,3,4,5,6,7,1,5,24,14,54,12,25])
idx = find_nearest(a,13)
a[idx]

14

# Concatination of multiple files
    -defining function concat_files

In [104]:
def find_stitch_idx(tus1=None, tus2=None, erg1=None, erg2=None):
    
    idx = find_nearest(tus1,tus2[0])
    if erg1[idx] != erg2[0]:
        
        idx_nearest_erg = find_nearest(erg1[idx:], erg2[0])
        stitch_idx = idx_nearest_erg + idx
        
        print("stitch_idx: ", stitch_idx)
        print("erg1[stitch_idx]", erg1[stitch_idx])
        print("erg2[0]", erg2[0])
        print(erg1[stitch_idx:] == erg2[0:len(erg1[stitch_idx:])])
        print("all ergs line up: ", np.all(erg1[stitch_idx:] == erg2[0:len(erg1[stitch_idx:])])) #Checking if the energies lines up
        
        print("erg1: ", erg1[stitch_idx:])
        print("erg2: ", erg2[0:len(erg1[13148:])])
        
        """    return stitch_idx
        else:
            print("Could not find a suitible index for stitching files")"""
    else:
        #found correct stich index
        return idx

In [107]:
def concat_GRB(grb_id, flag):
    """ function concat_files concatenates multiple matlab files. Can only concat HED or LED files at the time
    
    Flag determines whether the input list is of HED or LED files
    
    --> If there is "missing counts" between the files. 0 counts is added for the respective missing time interval. Automatically when time-shifted (?)
    
    Routine description: 
    1) The different tus arrays have their own T0. Calculating the delta T between T0 and Tn (time for concatination)  
    
    
    Parameters:
    %---------------------------------------------------------------
    
    grb_id <string>  : key for dictionary containing list of matlab filenames
    flag <string>    : "HED" or "LED". They have different arrays after calling read_matlab_file function. This flag handles that.
    
    
    returns:
    %---------------------------------------------------------------
    
    
    output for HED:
    %---------------------------------------------------------------
    concat_tus_HED <array> : concat tus (relative times in microseconds) for all arrays
    concat_erg_HED <array> : concat erg (energy channel (needs to be calibrated to keV using calibration coeff)) for all arrays
    concat_typ_HED <array> : concat typ for all counts  (normal (0), fast(1), valley (3), or overflow event (2 or 4 not )) read function keeps only 0 & 1.
    concat_adr_HED <array> : concat BGO address 1-12 for all arrays
    concat_dau_HED <array> : concat dau address 1-4
    concat_det_HED <array> : concat detector address 1-3
    concat_DetCol_HED <array> : concat RGB color for all arrays
    T0 <datetime> : datetime object of T0
    
    output for LED:
    %----------------------------------------------------------------
    
    
    """
    #Handling input is in the right format
    try:
        if type(flag) and type(grb_id) != str:
            raise TypeError("grb_id and/or flag input is not string")
        if flag != "HED" and flag != "LED":
            raise NameError("flag-string is neither HED or LED")
        if flag == "HED":
            if grb_id not in trig_dict_HED.keys():
                raise KeyError("GRB ID not in trig_dict_HED")
        if flag == "LED":
            if grb_id not in trig_dict_LED.keys():
                raise KeyError("GRB ID not in trig_dict_LED")
        else:
            if flag == "HED":
                HED_files = trig_dict_HED[grb_id] #list of HED filenames
                if all(HED_files[i] <= HED_files[i+1] for i in range(len(HED_files) - 1)) == False: #logic for checking that the files are in increasing order
                    raise IndexError("List is not sorted in increasing order")
            if flag == "LED":
                LED_files = trig_dict_LED[grb_id] #list of LED filenames in list
                if all(LED_files[i] <= LED_files[i+1] for i in range(len(LED_files) - 1)) == False: #logic for checking that the files are in increasing order
                    raise IndexError("List is not sorted in increasing order")
                    
                    
    except (TypeError, NameError, KeyError) as e:
        return print(e) #routine quits if error is found
    
    except IndexError:
        if flag == "HED":
            HED_files.sort() #sorting filenames in increasing order. Should already be done in dict creation, but good to check
        else:
            LED_files.sort() #sorting filenames in increasing order. Should already be done in dict creation, but good to check
            
            
    try: 
        data = [] #list that stores dictionary for each file
        if flag == "HED":
            
            num_files = len(HED_files)
            
            for filename in HED_files:
                
                tus, erg, typ, adr, dau, det, DetCol, date = read_matlab_file_with_BGO_data(filename)
                if len(date) != 27:
                    raise ValueError("length of datetime string is not in correct format")
                
                index_sort = np.argsort(tus) # sorting tus in increasing order
                print("length of index_sort: ", len(index_sort))
                tus_sort, erg_sort, typ_sort, adr_sort, dau_sort, det_sort = tus[index_sort], erg[index_sort], typ[index_sort], adr[index_sort],dau[index_sort], det[index_sort] #Making sure that the are coherent
                
                file_dict = {"tus":tus_sort, "erg":erg_sort, "typ":typ_sort, "adr":adr_sort, "dau":dau_sort, "det": det_sort, "DetCol":DetCol, "date":date}
                
                data.append(file_dict)
            
            T0 = datetime.strptime(data[0]["date"], "%Y-%b-%d %H:%M:%S.%f") #T0 of first file. Will be T0 for concat tus array

            # Adding the timeshift to all tus from T0
            for i in np.arange(1,num_files):
                deltaT = datetime.strptime(data[i]["date"], "%Y-%b-%d %H:%M:%S.%f") - T0
                print("deltaT: ",deltaT)
                data[i]["tus"] = data[i]["tus"] + float(deltaT.total_seconds())*1e6 #adding total microseconds to all entries
            
            return data
            
            """
            # Cutting arrays at respective index; idx
            i = 0
            while i < num_files-1: #Need num_files-1 cutting indecies
                print("filenum: ",i)
                idx = find_stitch_idx(tus1=data[i]["tus"], tus2=data[i+1]["tus"], erg1= data[i]["erg"] , erg2 = data[i+1]["erg"]) #index for cutting

            
            

                print("idx: ", idx)
                data[i]["tus"] = data[i]["tus"][:idx] #cutting arrays at idx
                data[i]["erg"] = data[i]["erg"][:idx] 
                data[i]["typ"] = data[i]["typ"][:idx]
                data[i]["dau"] = data[i]["dau"][:idx]
                
                i += 1 
            #DEBUG RETURN
            return data[0]["tus"],data[1]["tus"], data[0]["erg"], data[1]["erg"]
            #Concatenating respective arrays
            concat_tus_HED = np.concatenate([data[i]["tus"] for i in range(0,num_files)]) #Just concatinating tus arrays 
            concat_erg_HED = np.concatenate([data[i]["erg"] for i in range(0,num_files)])
            concat_typ_HED = np.concatenate([data[i]["typ"] for i in range(0,num_files)])
            concat_dau_HED = np.concatenate([data[i]["dau"] for i in range(0,num_files)])
            
            return concat_tus_HED, concat_erg_HED, concat_typ_HED, concat_dau_HED, DetCol, T0
        
        #LED concatination
        else: 
            
            for filename in LED_files:
                
                tus, erg, energy_keV, tus_keep_multihit, erg_keep_multihit, multihit_indx, date = read_matlab_file_with_CZT_data(filename)
                if len(date) != 27:
                    raise IndexError("length of datetime string is not in correct format of length 27")
                
                file_dict = {"tus":tus, "tus_keep_multihit": tus_keep_multihit, "erg":erg, "erg_keep_multihit": erg_keep_multihit, 
                             "energy_keV": energy_keV, "multihit_indx": multihit_indx, "date":date} #orginizing variables in dictionary
                
                data.append(file_dict) #dict containing variables stored in list. The index of the list determines which file. Allways in increasing order """
            
            
    except (ValueError) as e:
        return print(e)
    
    except (IndexError):
        idx = find_stitch_idx(tus1=data[i]["tus"], tus2=data[i]["tus"], erg1=data[0]["erg"] , erg2=ata[1]["erg"]) 
    
    print("no errors found")


## Testing concat

In [14]:
#concat_tus_HED, concat_erg_HED, concat_typ_HED, concat_dau_HED, DetCol, T0 = concat_GRB("GRB191221","HED")
#Deb

In [91]:
print(np.all(tus1 == tus1_ex))
print(np.all(tus2 == tus2_ex))
print(np.all(erg1 == erg1_ex)) #wrong
print(np.all(erg2 == erg2_ex))

True
True
False
True


In [95]:
mask = np.where(erg1!= erg1_ex)
len(mask[0])

1179

In [108]:
data=concat_GRB("GRB190825","HED")

length of index_sort:  13481
length of index_sort:  13556
deltaT:  0:00:01.999000


In [123]:
np.all(data[0]["erg"] == erg1_sort)

/tmp/ipykernel_134062/813002758.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.all(data[0]["erg"] == erg1_sort)


False

In [60]:
print(erg1[13148:] == erg2[0:len(erg1[13148:])])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [72]:
tus1, erg1, typ1, adr1, dau1, det1, DetCol1, date1 = read_matlab_file_with_BGO_data("2019_08_25_21_04_53")

In [167]:
tus1_ex, erg1_ex, typ1_ex, adr1, dau1, det1, DetCol1, date1 = read_matlab_file_with_BGO_data("2019_08_25_21_04_53")
tus1_ex, erg1_ex, typ1_ex, adr1, dau1, det1 = zip(*sorted(zip(tus1, erg1, typ1, adr1, dau1, det1))) #Sorted and coherent arrays

tus1_ex = np.asarray(tus1_ex)
erg1_ex = np.asarray(erg1_ex)
adr1 = np.asarray(adr1)
dau1 = np.asarray(dau1)
det1 = np.asarray(det1)

tus2_zip, erg2_zip, typ2, adr2, dau2, det2, DetCol2, date2  = read_matlab_file_with_BGO_data("2019_08_25_21_04_55")
tus2_zip, erg2_zip, typ2, adr2, dau2, det2  = zip(*sorted(zip(tus2, erg2, typ2, adr2, dau2, det2))) #Sorted and coherent arrays

tus2_zip = np.asarray(tus2_ex)
erg2_zip = np.asarray(erg2_ex)
adr2 = np.asarray(adr2)
dau2 = np.asarray(dau2)
det = np.asarray(det2)

T0 = datetime.strptime(date1, "%Y-%b-%d %H:%M:%S.%f")
T1 = datetime.strptime(date2, "%Y-%b-%d %H:%M:%S.%f")
deltaT = T1 - T0

tus2_zip = tus2_zip + deltaT.total_seconds()*1e6


In [163]:
tus1_ex, erg1_ex, typ1_ex, adr1, dau1, det1, DetCol1, date1 = read_matlab_file_with_BGO_data("2019_08_25_21_04_53")
index_sort1 = np.argsort(tus1_ex) # sorting tus in increasing order
              
tus1_sort = tus1_ex[index_sort1]
erg1_sort = erg1_ex[index_sort1] #Making sure that the are coherent




tus2_ex, erg2_ex, typ_ex, adr1, dau1, det1, DetCol2, date2 = read_matlab_file_with_BGO_data("2019_08_25_21_04_55")
index_sort2 = np.argsort(tus2_ex) # sorting tus in increasing order
              
tus2_sort_np, erg2_sort_np = tus2_ex[index_sort2], erg2_ex[index_sort2] #Making sure that the are coherent

T0 = datetime.strptime(date1, "%Y-%b-%d %H:%M:%S.%f")
T1 = datetime.strptime(date2, "%Y-%b-%d %H:%M:%S.%f")

deltaT = T1 - T0

tus2_ex_np = tus2_sort + deltaT.total_seconds()*1e6


In [168]:
np.where(erg2_sort_np != erg2_zip)

(array([   50,    51,    64,    65,    76,    78,   170,   171,   195,
          196,   234,   235,   261,   262,   314,   315,   316,   340,
          341,   460,   461,   474,   475,   646,   647,   672,   673,
          685,   686,   687,   858,   859,  1002,  1003,  1004,  1135,
         1136,  1137,  1138,  1320,  1321,  1322,  1323,  1386,  1387,
         1388,  1398,  1399,  1481,  1482,  1758,  1759,  1771,  1772,
         1943,  1944,  2024,  2025,  2026,  2032,  2033,  2034,  2035,
         2116,  2117,  2169,  2170,  2235,  2236,  2525,  2526,  2553,
         2554,  2592,  2593,  2604,  2605,  2685,  2686,  2923,  2924,
         2977,  2978,  3123,  3124,  3335,  3336,  3362,  3363,  3364,
         3373,  3374,  3493,  3494,  3624,  3625,  3716,  3717,  3719,
         3720,  3770,  3771,  3781,  3782,  3836,  3837,  3890,  3891,
         3902,  3903,  3904,  3970,  3971,  3996,  3998,  4168,  4169,
         4195,  4196,  4247,  4248,  4260,  4261,  4300,  4301,  4363,
      

In [148]:
for i in mask:
    print(tus1_sort[i])
    print(tus1_ex[i])

[-45087.86111111 -45055.08333333 -44799.16666667 -44645.25
 -42786.88888889 -42539.25       -41130.33333333 -40869.63888889
 -40594.66666667 -33733.94444444 -32986.91666667 -28822.13888889
 -28822.13888889 -27731.47222222 -27731.47222222 -27143.97222222
 -26882.19444444 -25766.72222222 -24623.44444444 -23808.75
 -23757.72222222 -19774.22222222 -19541.75       -19305.
 -18768.11111111 -18768.05555556 -13427.5        -13427.41666667
  -9875.05555556  -9448.11111111  -5639.30555556  -5560.22222222
  -5245.27777778  -5157.22222222  -4957.88888889]
[-45087.86111111 -45055.08333333 -44799.16666667 -44645.25
 -42786.88888889 -42539.25       -41130.33333333 -40869.63888889
 -40594.66666667 -33733.94444444 -32986.91666667 -28822.13888889
 -28822.13888889 -27731.47222222 -27731.47222222 -27143.97222222
 -26882.19444444 -25766.72222222 -24623.44444444 -23808.75
 -23757.72222222 -19774.22222222 -19541.75       -19305.
 -18768.11111111 -18768.05555556 -13427.5        -13427.41666667
  -9875.0555555

In [149]:
for i in mask:
    print(erg1_sort[i])
    print(erg1_ex[i])

[  19   41   66 1365   36   30  994  470   54 1671   36  255   31 1681
 1747  107  109   82   45   31   60  127  136  147   36  191   74   23
   54   16   29  159   48  440   38]
[  19   41   66 1365   36   30  994  470   54 1671   36   28   31 1866
 1906  107  109   82   45   31   60  127  136  147   36  191   74   23
   54   16   29  159   48  440   38]


In [141]:
idx = find_stitch_idx(tus1=tus1_ex, tus2=tus2_ex, erg1=erg1_ex, erg2=erg2_ex)

stitch_idx:  13148
erg1[stitch_idx] 1584
erg2[0] 1584
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  Tr

In [138]:
len(tus2_sort)

13556

In [103]:
erg2_ex[0:len(tus1_ex[13148:])]

array([1584,   70,   62,   42,  168,   73,  102, 1716,   26,  151,   75,
       1694,  121, 1538,   53,   61,   34,   66,   32, 1233, 1675,   86,
        332,  221,   21,   67, 1973,   33,   41, 1828,   36,   65,   31,
         43,   31,   74,   40,   26, 1626,  407,  285,  171,   49,  196,
       2055, 1735,   81,   56,  143,  295,   36,  218,   76,  175,  192,
        101,   50,  983,   46,   41,   41,   62,  150,   27,  257,   66,
         61,  168,   34,  156,   35,  487,  102,   17,   33,  179,  265,
        152,  152,  273,  628, 1071,   55,   54,   40, 1666, 1781,  347,
        102,  178, 1655,  629,   97,   26,   48,   18,  122,   34,   43,
       1428, 1721,   41,   51,  125,   39,   76,   41,   88,   80,   34,
         26,  166,   55,  406,   45,  123,   22, 1601,   36, 1361,   21,
        696,   43,  492,   22,   37,   34,   70,  265,   27,  475,   29,
       1568, 1655, 1515, 1754,   16,   57, 1310,  142,  195,   32,  183,
        838,   40,   29,   86,   46,  252,   22,  1

In [66]:
print(erg2[0:len(erg1[13148:])])

[1584   70   62   42  168   73  102 1716   26  151   75 1694  121 1538
   53   61   34   66   32 1233 1675   86  332  221   21   67 1973   33
   41 1828   36   65   31   43   31   74   40   26 1626  407  285  171
   49  196 2055 1735   81   56  143  295   36  218   76  175  192  101
   50  983   46   41   41   62  150   27   66  257   61  168   34  156
   35  487  102   17   33  179  152  152  265  273  628 1071   55   54
   40 1666 1781  102  178  347 1655  629   97   26   48   18  122   34
   43 1428 1721   41   51  125   39   76   41   88   80   34   26  166
   55  406   45  123   22 1601   36 1361   21  696   43  492   22   37
   34   70  265   27  475   29 1568 1515 1655 1754   16   57 1310  142
  195   32  183  838   40   29   86   46   22  252  193 2015 1507  230
   19  154 1664  164   50  653 1857   51   28   77   38   49   33   93
   39 1412   22   76   36 1452   44   27  108 1821   64 1630 1515 1543
   96   36  253  367  497   36   24  139   61  355   99  384   28 1766
 1923 

In [16]:
np.argsort()
a = np.array(["a","c","b"])
b = np.array([1,3,2])


In [17]:
a, b

(array(['a', 'c', 'b'], dtype='<U1'), array([1, 3, 2]))

In [18]:
index_sort = np.argsort(b)
a = a[index_sort]
b = b[index_sort]

In [19]:
a, b

(array(['a', 'b', 'c'], dtype='<U1'), array([1, 2, 3]))

In [ ]:
tus1[13139] - tus2_shift[0]

In [ ]:
tus2_shift[0]

# main function

In [ ]:
def main(grb_id,binsize=10000, start_GRB, end_GRB):  # filename GRB ID
    # output array for tus_keep_multihit_LED, erg_keep_multihit_LED, multihit_indx_LED må modifiseres
    # Automake by making filename a list of filenames. Then the all the outout plots will be created

    # TODO:
    # Add the extra parameters to make the get_mean work
#os.mkdir("/Users/andreas/phys/master/MATLAB/pyplots")
    trig_dict = create_triggers_dict()
    try:
        output_dir = os.mkdir("/Home/siv30/wad005/master/GRB/MATLAB/pyplots/{}".format(grb_id)) #Creating owd folder for grb_id
    except FileExistsError:
        output_dir = "/Home/siv30/wad005//master/GRB/MATLAB/pyplots/{}".format(grb_id) #selecting path if it already exists


    if type(grb_id) != str:
        raise TypeError("Input is not string")
    elif grb_id not in trig_dict:
        raise LookupError("grb_id not in dictionary")
    else:
        try:
            LED_filename = trig_dict[grb_id][0]
            HED_filename = trig_dict[grb_id][1]
            if type(LED_filename) == float and math.isnan(LED_filename):
                print("LED file not found. Plotting only HED")
                plot_HED(HED_filename,output_dir,grb_id) #output path is not required
                
            elif type(HED_filename) == float and math.isnan(HED_filename):
                print("HED file not found. Plotting only LED")
                plot_LED(LED_filename,output_dir,grb_id)
            else:
                plot_LED(LED_filename,output_dir,grb_id)
                plot_HED(HED_filename,output_dir,grb_id)
        except:
            print("Something went wrong")
    
    

# Graveyards

In [ ]:
# Not in use
def get_filenames(path):
    return sorted((f for f in os.listdir(path) if not f.startswith(".")), key=str.lower)

In [ ]:
#Old plot_HED function
"""
def plot_HED(HED_filename, output_dir,grb_id):
    tus_HED, erg_HED, typ_HED, adr_HED, dau_HED, det_HED, DetCol_HED = read_matlab_file_with_BGO_data(HED_filename)

    fig0, (ax0, ax1) = plt.subplots(
        2, 1, figsize=(8, 8), sharex=True, sharey=False)
    ax0.set_title("HED")

    ax0.scatter(tus_HED, erg_HED)
    ax0.set_xlabel("time [us]")
    ax0.set_ylabel("Energy channel")

    binsize = 10000  # us --> 10 ms bin
    ax1.hist(tus_HED, bins=np.arange(
        min(tus_HED), max(tus_HED)+binsize, binsize))
    ax1.set_xlabel("time [us]")
    ax1.set_ylabel("Counts per bin")
    
    output_dir_hed = output_dir + "/{}_HED".format(grb_id)
    print(output_dir_hed)
    plt.savefig(output_dir + "/{}_HED".format(grb_id)) #path where plot is saved
"""